## Kenh 14

In [2]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')

# thiết lập đường dẫn file
def setup_path():
    """Thiết lập đường dẫn lưu file CSV."""
    output_path = "/content/drive/MyDrive/Scrapper_Kenh14_update.csv"
    return output_path

# crawl dữ liệu trang báo
def fetch_main_page():
  """fetch the main page (kenh 14)"""
  url = 'http://kenh14.vn'
  try:
    response = requests.get(url, timeout = 5)
    response.raise_for_status()
    return response.text, url
  except requests.exceptions.RequestException as e:
    print ({f"Error matching main page: {e}"})
    return None, url

# Lấy danh sách chuyên mục
def fetch_menu_links(url, headers):
    """Lấy danh sách chuyên mục từ trang Kênh 14."""
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    menu = soup.find("ul", class_="kbh-menu-list clearfix fl")
    menu_links = {}

    if menu:
        for item in menu.find_all("li"):
            link_tag = item.find("a")
            if link_tag:
                text = link_tag.get_text(strip=True)
                link = link_tag["href"]
                if link.startswith("/"):
                    link = url + link.lstrip("/")
                menu_links[text] = link
    return menu_links

# Lấy bài viết của từng chuyên mục
def crawl_category_page(category_name, category_url, headers, url):
    """Trích xuất danh sách bài viết từ chuyên mục."""
    print(f"Đang crawl: {category_name} - {category_url}")
    article_data = []

    if category_url.startswith("http"):
        response = requests.get(category_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all("h3", class_="knswli-title")

        for article in articles:
            a_tag = article.find("a")
            if a_tag:
                article_id = a_tag.get("newsid", None)
                title = a_tag.get_text(strip=True)
                href = a_tag["href"]
                if href.startswith("/"):
                    href = url + href.lstrip("/")
                article_data.append({
                    "ID": article_id,
                    "Title": title,
                    "Href": href,
                    "Category": category_name
                })
    else:
        print(f" - Bỏ qua URL không hợp lệ: {category_url}")

    return article_data

# Lấy nội dung của từng bài viết trong chuyên mục
def extract_article_content(df, headers):
    """Lấy nội dung bài viết và thời gian đăng bài."""
    df["Content"] = None
    df["Time"] = None

    for index, row in df.iterrows():
        article_url = row['Href']
        try:
            response = requests.get(article_url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # Lấy nội dung bài viết
            content_div = soup.find("div", class_="detail-content afcbc-body")
            content = content_div.get_text(separator="\n", strip=True) if content_div else "Không tìm thấy nội dung"

            # Lấy thời gian đăng bài
            time_div = soup.find("div", class_="kbwc-meta")
            time_span = time_div.find("span", class_="kbwcm-time") if time_div else None
            time = time_span.get_text(strip=True) if time_span else "Không tìm thấy thời gian"

            df.at[index, "Content"] = content
            df.at[index, "Time"] = time

        except requests.exceptions.RequestException as e:
            print(f"❌ Lỗi khi lấy {article_url}: {e}")

    return df

def main():
    """Hàm chính để chạy toàn bộ quá trình."""
    output_path = setup_path()
    url = "https://kenh14.vn/"
    headers = {"User-Agent": "Mozilla/5.0"}

    # Lấy danh sách chuyên mục
    menu_links = fetch_menu_links(url, headers)

    # Crawl dữ liệu từ từng chuyên mục
    all_articles = []
    for category, link in menu_links.items():
        articles = crawl_category_page(category, link, headers, url)
        all_articles.extend(articles)

    # Chuyển dữ liệu thành DataFrame
    df = pd.DataFrame(all_articles)

    # Lấy nội dung và thời gian đăng bài
    df = extract_article_content(df, headers)

    # Lưu vào file CSV
    file_exists = os.path.exists(output_path)
    df.to_csv(output_path, mode="a", header=not file_exists, index=False)

    print("✅ Đã lưu dữ liệu vào:", output_path)
    return df

if __name__ == "__main__":
    data_kenh14 = main()
    if data_kenh14 is not None:
        print(data_kenh14.head(5))

Mounted at /content/drive
Đang crawl: TRANG CHỦ - https://kenh14.vn/
Đang crawl: Star - https://kenh14.vn/star.chn
Đang crawl: Ciné - https://kenh14.vn/cine.chn
Đang crawl: Musik - https://kenh14.vn/musik.chn
Đang crawl: Beauty & Fashion - https://kenh14.vn/beauty-fashion.chn
Đang crawl: Đời sống - https://kenh14.vn/doi-song.chn
Đang crawl: Money-Z - https://kenh14.vn/money-z.chn
Đang crawl: Ăn - Quẩy - Đi - https://kenh14.vn/an-quay-di.chn
Đang crawl: Xã hội - https://kenh14.vn/xa-hoi.chn
Đang crawl: Sức khỏe - https://kenh14.vn/suc-khoe.chn
Đang crawl: Tek-life - https://kenh14.vn/tek-life.chn
Đang crawl: Học đường - https://kenh14.vn/hoc-duong.chn
Đang crawl: Xem Mua Luôn - https://kenh14.vn/xem-mua-luon.chn
Đang crawl: Video - http://video.kenh14.vn/
Đang crawl:  - javascript:;
 - Bỏ qua URL không hợp lệ: javascript:;
Đang crawl: Nhân vật - https://kenh14.vn/hoc-duong/nhan-vat.chn
Đang crawl: Xem-Ăn-Chơi - https://kenh14.vn/xem-an-choi.chn
Đang crawl: House n Home - https://kenh14.

In [3]:
data_kenh14

,ID,Title,Href,Category,Content,Time
0,215250613121224562,Ta là ai trong thời đại A.I?,https://kenh14.vn/ta-la-ai-trong-thoi-dai-ai-2...,TRANG CHỦ,Không tìm thấy nội dung,Không tìm thấy thời gian
1,None,"Đăng quang Hoa hậu Việt Nam, Hà Trúc Linh sẽ k...",https://kenh14.vn/dang-quang-hoa-hau-viet-nam-...,Star,Không tìm thấy nội dung,14:28 08/07/2025
2,None,Tân hoa hậu Việt vừa đăng quang đã vướng tranh...,https://kenh14.vn/tan-hoa-hau-viet-vua-dang-qu...,Star,"Chỉ trong tầm 1 tháng gần đây, showbiz Việt có...",14:22 08/07/2025
3,None,Hai nhóc tỳ nhà Phương Oanh - shark Bình khiến...,https://kenh14.vn/hai-nhoc-ty-nha-phuong-oanh-...,Star,"Sau thời gian kín tiếng về đời tư, diễn viên Đ...",14:00 08/07/2025
4,None,Dâu trưởng bất trị của Beckham giờ gây chiến đ...,https://kenh14.vn/dau-truong-bat-tri-cua-beckh...,Star,Mối quan hệ giữa vợ chồng\nBrooklyn Beckham\n-...,12:34 08/07/2025
...,...,...,...,...,...,...
344,None,Ba thói quen “phá mạch máu” nhanh hơn ăn thịt ...,https://kenh14.vn/ba-thoi-quen-pha-mach-mau-nh...,Các bệnh,Mạch máu là một phần quan trọng của hệ tuần ho...,10:57 07/07/2025
345,None,"Sau khi ăn món khoái khẩu, người đàn ông 73 tu...",https://kenh14.vn/sau-khi-an-mon-khoai-khau-ng...,Các bệnh,Một vụ ngộ độc hi hữu vừa xảy ra tại thành phố...,10:32 07/07/2025
346,None,Nam kỹ sư trẻ đi khám tâm thần vì tim đập nhan...,https://kenh14.vn/nam-ky-su-tre-di-kham-tam-th...,Các bệnh,"Gần năm nay, nam thanh niên liên tục gặp các t...",09:36 07/07/2025
347,None,Đừng ăn 3 thực phẩm này mỗi ngày kẻo tổn thươn...,https://kenh14.vn/dung-an-3-thuc-pham-nay-moi-...,Các bệnh,Nồng độ axit uric cao không chỉ gây ra bệnh gú...,17:06 06/07/2025


### KNN

### LogisticRegression

## Báo tuổi trẻ

In [6]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')

def setup_path():
    """Set up file path for output"""
    ouput_path = "/content/drive/MyDrive/Scrapper_baotuoitre_update.csv"
    return ouput_path

def fetch_main_page():
    """fetch main page form baotuoitre"""
    base_url = "https://tuoitre.vn/"
    try:
      response = requests.get(base_url, timeout=5)
      response.raise_for_status()
      return response.text, base_url
    except requests.exceptions.RequestException as e:
      print ({f"Error fetching main page: {e}"})
      return None, base_url

from urllib.parse import urljoin

def fetch_menu_links(url, headers):
    """Lấy danh sách chuyên mục từ trang Báo tuổi trẻ."""
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    menu = soup.find("ul", class_="menu-nav")
    menu_links = {}

    if menu:
        for item in menu.find_all("li"):
            link_tag = item.find("a")
            if link_tag:
                text = link_tag.get_text(strip=True)
                link = link_tag["href"]
                # Assign link to href directly instead of using a separate variable
                href = urljoin(url, link)
                menu_links[text] = href  # Use href, which has the joined URL
    return menu_links

def crawl_category_page(category_name, category_url, headers, url):
    """Trích xuất danh sách bài viết từ chuyên mục."""
    print(f"📰 Đang crawl: {category_name} - {category_url}")
    article_data = []

    if category_url.startswith("http"):
        try:
            response = requests.get(category_url, headers=headers)
            response.raise_for_status()
        except Exception as e:
            print(f"❌ Không thể tải trang {category_url}: {e}")
            return article_data

        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.find_all("h3", class_="box-title-text")

        for article in articles:
            a_tag = article.find("a")
            if a_tag:
                article_id = a_tag.get("newsid")
                if not article_id:
                    article_id = hash(a_tag["href"])

                title = a_tag.get_text(strip=True)
                href = a_tag["href"]
                href = urljoin(url, href)

                article_data.append({
                    "ID": str(article_id),
                    "Title": title,
                    "Href": href,
                    "Category": category_name  # ✅ Luôn thêm
                })

        print(f"  → Tìm thấy {len(article_data)} bài viết")
    else:
        print(f"⚠️ Bỏ qua URL không hợp lệ: {category_url}")

    return article_data


# Lấy nội dung của từng bài viết trong chuyên mục
def extract_article_content(df, headers):
    """Lấy nội dung bài viết và thời gian đăng bài."""
    df["Content"] = None
    df["Time"] = None

    for index, row in df.iterrows():
        article_url = row['Href']
        try:
            response = requests.get(article_url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            # Lấy nội dung bài viết
            content_div = soup.find("div", class_="detail-content afcbc-body")
            content = content_div.get_text(separator="\n", strip=True) if content_div else "Không tìm thấy nội dung"

            # Lấy thời gian đăng bài
            time_div = soup.find("div", class_="detail-time")
            time = time_div.get_text(strip = True) if time_div else "Không tìm thấy thời gian"

            df.at[index, "Content"] = content
            df.at[index, "Time"] = time

        except requests.exceptions.RequestException as e:
            print(f"❌ Lỗi khi lấy {article_url}: {e}")

    return df

def main():
    """Hàm chính để chạy toàn bộ quá trình."""
    output_path = setup_path()
    url = "https://tuoitre.vn/" # Define 'url' here
    headers = {"User-Agent": "Mozilla/5.0"}

    # Lấy danh sách chuyên mục
    menu_links = fetch_menu_links(url, headers)

    # Crawl dữ liệu từ từng chuyên mục
    all_articles = []
    for category, link in menu_links.items():
        articles = crawl_category_page(category, link, headers, url) # Pass 'url' to the function
        all_articles.extend(articles)

    # Chuyển dữ liệu thành DataFrame
    df = pd.DataFrame(all_articles)
    df["ID"] = df["ID"].astype(str)

    # Lấy nội dung và thời gian đăng bài
    df = extract_article_content(df, headers)

    # Lưu vào file CSV
    file_exists = os.path.exists(output_path)
    df.to_csv(output_path, mode="a", header=not file_exists, index=False)

    print("✅ Đã lưu dữ liệu vào:", output_path)
    return df

if __name__ == "__main__":
    data_baotuoitre = main()
    if data_baotuoitre is not None:
        print(data_baotuoitre.head(5))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📰 Đang crawl:  - https://tuoitre.vn/
  → Tìm thấy 109 bài viết
📰 Đang crawl: Video - https://tuoitre.vn/video.htm
  → Tìm thấy 74 bài viết
📰 Đang crawl: Thời sự - https://tuoitre.vn/thoi-su.htm
  → Tìm thấy 20 bài viết
📰 Đang crawl: Thế giới - https://tuoitre.vn/the-gioi.htm
  → Tìm thấy 20 bài viết
📰 Đang crawl: Pháp luật - https://tuoitre.vn/phap-luat.htm
  → Tìm thấy 17 bài viết
📰 Đang crawl: Kinh doanh - https://tuoitre.vn/kinh-doanh.htm
  → Tìm thấy 20 bài viết
📰 Đang crawl: Công nghệ - https://tuoitre.vn/cong-nghe.htm
  → Tìm thấy 29 bài viết
📰 Đang crawl: Xe - https://tuoitre.vn/xe.htm
  → Tìm thấy 29 bài viết
📰 Đang crawl: Du lịch - https://tuoitre.vn/du-lich.htm
  → Tìm thấy 20 bài viết
📰 Đang crawl: Nhịp sống trẻ - https://tuoitre.vn/nhip-song-tre.htm
  → Tìm thấy 29 bài viết
📰 Đang crawl: Văn hóa - https://tuoitre.vn/van-hoa.htm
  → Tìm thấy 23 bài

In [7]:
data_baotuoitre.head(10)

,ID,Title,Href,Category,Content,Time
0,-2260756287483343458,Tăng trưởng tín dụng cả nước gần 10%: cao nhất...,https://tuoitre.vn/tang-truong-tin-dung-ca-nuo...,,Phó thống đốc Ngân hàng Nhà nước Phạm Thanh Hà...,08/07/2025 15:15 GMT+7
1,682725038049155215,Loạt xe điện chở khách du lịch bị thiêu rụi tr...,https://tuoitre.vn/loat-xe-dien-cho-khach-du-l...,,Ngọn lửa bốc cháy dữ dội tại nhà để xe điện - ...,08/07/2025 15:12 GMT+7
2,4992644287564071236,Tác dụng đặc biệt ít ai biết về củ khoai lang,https://tuoitre.vn/tac-dung-dac-biet-it-ai-bie...,,Khoai lang có nhiều tác dụng đặc biệt - Ảnh mi...,08/07/2025 15:16 GMT+7
3,1068133380818326384,Lũ lụt kinh hoàng ở Texas bước sang ngày thứ 5...,https://tuoitre.vn/lu-lut-kinh-hoang-o-texas-b...,,Hình ảnh mực nước con sông Guadalupe ở hạt Ker...,08/07/2025 15:20 GMT+7
4,1159546553898035122,"Cần thủ vào mùa săn cá cóc trên sông Hậu, tỉnh...",https://tuoitre.vn/can-thu-vao-mua-san-ca-coc-...,,Anh Nguyễn Thanh Bình cùng số cá cóc câu được ...,08/07/2025 15:19 GMT+7
5,1125808989975766984,Đường dây tống tiền bằng hình ảnh nhạy cảm của...,https://tuoitre.vn/duong-day-tong-tien-bang-hi...,,Những người liên quan đến đường dây tống tiền ...,08/07/2025 13:53 GMT+7
6,-2142646541881351185,"Cháy nhiều phòng trọ ở phường Tân Thới Hiệp, T...",https://tuoitre.vn/chay-nhieu-phong-tro-o-phuo...,,Cảnh sát phòng cháy chữa cháy đến hiện trường ...,08/07/2025 11:11 GMT+7
7,6128318560092971323,"Thái Lan, Nhật Bản và nhiều nước lên tiếng về ...",https://tuoitre.vn/thai-lan-nhat-ban-va-nhieu-...,,Tổng thống Donald Trump ngày 6-7 - Ảnh: AFP\n...,08/07/2025 13:42 GMT+7
8,-3099726701672378505,Bí thư Thành ủy TP.HCM Nguyễn Văn Nên viếng cá...,https://tuoitre.vn/bi-thu-thanh-uy-tp-hcm-nguy...,,"Bí thư Nguyễn Văn Nên chia sẻ, động viên gia đ...",08/07/2025 11:47 GMT+7
9,-3774541804536343103,Không dự đám tang nhưng Cristiano Ronaldo đã đ...,https://tuoitre.vn/khong-du-dam-tang-nhung-cri...,,Hình ảnh cho thấy cầu thủ Cristiano Ronaldo vi...,08/07/2025 09:22 GMT+7


In [8]:
data_baotuoitre['Category'] = data_baotuoitre['Category'].replace('', 'Trang chủ')
data_baotuoitre

,ID,Title,Href,Category,Content,Time
0,-2260756287483343458,Tăng trưởng tín dụng cả nước gần 10%: cao nhất...,https://tuoitre.vn/tang-truong-tin-dung-ca-nuo...,Trang chủ,Phó thống đốc Ngân hàng Nhà nước Phạm Thanh Hà...,08/07/2025 15:15 GMT+7
1,682725038049155215,Loạt xe điện chở khách du lịch bị thiêu rụi tr...,https://tuoitre.vn/loat-xe-dien-cho-khach-du-l...,Trang chủ,Ngọn lửa bốc cháy dữ dội tại nhà để xe điện - ...,08/07/2025 15:12 GMT+7
2,4992644287564071236,Tác dụng đặc biệt ít ai biết về củ khoai lang,https://tuoitre.vn/tac-dung-dac-biet-it-ai-bie...,Trang chủ,Khoai lang có nhiều tác dụng đặc biệt - Ảnh mi...,08/07/2025 15:16 GMT+7
3,1068133380818326384,Lũ lụt kinh hoàng ở Texas bước sang ngày thứ 5...,https://tuoitre.vn/lu-lut-kinh-hoang-o-texas-b...,Trang chủ,Hình ảnh mực nước con sông Guadalupe ở hạt Ker...,08/07/2025 15:20 GMT+7
4,1159546553898035122,"Cần thủ vào mùa săn cá cóc trên sông Hậu, tỉnh...",https://tuoitre.vn/can-thu-vao-mua-san-ca-coc-...,Trang chủ,Anh Nguyễn Thanh Bình cùng số cá cóc câu được ...,08/07/2025 15:19 GMT+7
...,...,...,...,...,...,...
552,-5994450664317358316,"Bão số 2 mạnh lên cấp 9, giật cấp 11",https://tuoitre.vn/bao-so-2-manh-len-cap-9-gia...,THỜI TIẾT,Dự báo vị trí và hướng di chuyển bão số 2 lúc ...,05/07/2025 11:40 GMT+7
553,-1966929887162257846,Thời tiết đêm 7 ngày 8-7: Bắc Bộ và Trung Bộ n...,https://tuoitre.vn/video/thoi-tiet-dem-7-ngay-...,THỜI TIẾT,,07/07/2025
554,-7442233498374739812,Thời tiết đêm 6 ngày 7-7: Bão số 2 sức gió m...,https://tuoitre.vn/video/thoi-tiet-dem-6-ngay-...,THỜI TIẾT,,06/07/2025
555,3663470927944687233,Thời tiết đêm 5 ngày 6-7: Bão số 2 ở bắc Biê...,https://tuoitre.vn/video/thoi-tiet-dem-5-ngay-...,THỜI TIẾT,,05/07/2025


## VnExppress

In [9]:
import pandas as pd
import requests
import os
from bs4 import BeautifulSoup
from google.colab import drive
import time

drive.mount('/content/drive')

def setup_path():
    """Thiết lập đường dẫn lưu file CSV."""
    output_path = "/content/drive/MyDrive/Scrapper_vnexpress_update.csv"
    return output_path

def fetch_main_page():
    """Fetch the main page (VnExpress)"""
    url = 'https://vnexpress.net/'
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return response.text, url
    except requests.exceptions.RequestException as e:
        print(f"Error fetching main page: {e}")
        return None, url

def fetch_menu_links(url, headers):
    """Lấy danh sách chuyên mục từ trang VnExpress."""
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        menu_links = {}

        # Tìm menu chính - cập nhật selector theo cấu trúc hiện tại
        main_menu = soup.find("nav", class_="main-nav") or soup.find("div", class_="menu-main")

        if main_menu:
            # Lấy các mục menu chính
            for item in main_menu.find_all("a", class_="parent", recursive=True):
                text = item.get_text(strip=True)
                link = item.get("href", "")
                if link.startswith("/"):
                    link = url + link.lstrip("/")
                if text and link:
                    menu_links[text] = link

        # Nếu không tìm thấy menu chính, thử cách khác
        if not menu_links:
            for item in soup.select(".main-nav a, .menu-main a"):
                text = item.get_text(strip=True)
                link = item.get("href", "")
                if link.startswith("/"):
                    link = url + link.lstrip("/")
                if text and link and text not in menu_links:
                    menu_links[text] = link

        return menu_links

    except Exception as e:
        print(f"Error fetching menu links: {e}")
        return {}

def crawl_category_page(category_name, category_url, headers, url):
    """Trích xuất danh sách bài viết từ chuyên mục."""
    print(f"Đang crawl: {category_name} - {category_url}")
    article_data = []

    try:
        if not category_url.startswith("http"):
            print(f" - Bỏ qua URL không hợp lệ: {category_url}")
            return article_data

        response = requests.get(category_url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        articles = soup.select("article.item-news, .list-news-subfolder .item-news, .title-news a")

        for article in articles:
            a_tag = article.find("a") if article.name != "a" else article
            if a_tag:
                title = a_tag.get("title", "") or a_tag.get_text(strip=True)
                href = a_tag.get("href", "")

                if href.startswith("/"):
                    href = url + href.lstrip("/")

                # Lấy ID bài viết từ URL hoặc data attribute
                article_id = href.split("-")[-1].split(".")[0] if href else ""

                if title and href:
                    article_data.append({
                        "ID": article_id,
                        "Title": title,
                        "Href": href,
                        "Category": category_name
                    })

        # Giới hạn số bài viết để tránh quá tải
        return article_data[:50]

    except Exception as e:
        print(f"Error crawling category {category_name}: {e}")
        return article_data

def extract_article_content(df, headers):
    """Lấy nội dung bài viết và thời gian đăng bài."""
    df["Content"] = None
    df["Time"] = None

    for index, row in df.iterrows():
        try:
            url_href = row["Href"]
            response = requests.get(url_href, headers=headers, timeout=10)
            soup_href = BeautifulSoup(response.text, "html.parser")

            # Lấy nội dung bài viết
            content_div = soup_href.find("article", class_="fck_detail") or soup_href.find("div", class_="content-detail")
            paragraphs = content_div.find_all("p") if content_div else []
            content = "\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])

            # Lấy thời gian đăng bài
            time_element = soup_href.find("span", class_="date") or soup_href.find("span", class_="time")
            article_time = time_element.get_text(strip=True) if time_element else "Không tìm thấy thời gian"

            # Cập nhật DataFrame
            df.at[index, "Time"] = article_time
            df.at[index, "Content"] = content

            # Tránh request quá nhanh
            time.sleep(1)

        except Exception as e:
            print(f"Error scraping article {row['Href']}: {e}")

    return df

def main():
    """Hàm chính để chạy toàn bộ quá trình."""
    output_path = setup_path()
    url = "https://vnexpress.net/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7"
    }

    # Lấy danh sách chuyên mục
    print("Đang lấy danh sách chuyên mục...")
    menu_links = fetch_menu_links(url, headers)
    print(f"Tìm thấy {len(menu_links)} chuyên mục")

    # Crawl dữ liệu từ từng chuyên mục
    all_articles = []
    for category, link in menu_links.items():
        articles = crawl_category_page(category, link, headers, url)
        all_articles.extend(articles)
        print(f" - Đã thu thập {len(articles)} bài viết từ chuyên mục {category}")

    # Chuyển dữ liệu thành DataFrame
    df = pd.DataFrame(all_articles)

    if not df.empty:
        # Lấy nội dung và thời gian đăng bài
        print("Đang trích xuất nội dung bài viết...")
        df = extract_article_content(df, headers)

        # Lưu vào file CSV
        file_exists = os.path.exists(output_path)
        df.to_csv(output_path, mode="a", header=not file_exists, index=False, encoding="utf-8-sig")

        print("✅ Đã lưu dữ liệu vào:", output_path)
        print(f"Tổng số bài viết: {len(df)}")
        return df
    else:
        print("Không tìm thấy bài viết nào")
        return None

if __name__ == "__main__":
    data_vnexpress = main()
    if data_vnexpress is not None:
        print("\n5 bài viết đầu tiên:")
        print(data_vnexpress.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Đang lấy danh sách chuyên mục...
Tìm thấy 20 chuyên mục
Đang crawl: Mới nhất - https://vnexpress.net/tin-tuc-24h
 - Đã thu thập 50 bài viết từ chuyên mục Mới nhất
Đang crawl: Thời sự - https://vnexpress.net/thoi-su
 - Đã thu thập 50 bài viết từ chuyên mục Thời sự
Đang crawl: Thế giới - https://vnexpress.net/the-gioi
 - Đã thu thập 50 bài viết từ chuyên mục Thế giới
Đang crawl: Kinh doanh - https://vnexpress.net/kinh-doanh
 - Đã thu thập 50 bài viết từ chuyên mục Kinh doanh
Đang crawl: Pháp luật - https://vnexpress.net/phap-luat
 - Đã thu thập 50 bài viết từ chuyên mục Pháp luật
Đang crawl: Khoa học công nghệ - https://vnexpress.net/khoa-hoc-cong-nghe
 - Đã thu thập 50 bài viết từ chuyên mục Khoa học công nghệ
Đang crawl: Thể thao - https://vnexpress.net/the-thao
 - Đã thu thập 50 bài viết từ chuyên mục Thể thao
Đang crawl: Giải trí - https://vnexpress.net/gia

In [10]:
data_vnexpress.head(20)

,ID,Title,Href,Category,Content,Time
0,4911600,Cựu phó tổng tham mưu trưởng Nga bị kết án tù ...,https://vnexpress.net/cuu-pho-tong-tham-muu-tr...,Mới nhất,,Không tìm thấy thời gian
1,4911600,Cựu phó tổng tham mưu trưởng Nga bị kết án tù ...,https://vnexpress.net/cuu-pho-tong-tham-muu-tr...,Mới nhất,,Không tìm thấy thời gian
2,4911596,200 trẻ có dấu hiệu nhiễm độc chì vì món bánh ...,https://vnexpress.net/200-tre-co-dau-hieu-nhie...,Mới nhất,,Không tìm thấy thời gian
3,4911596,200 trẻ có dấu hiệu nhiễm độc chì vì món bánh ...,https://vnexpress.net/200-tre-co-dau-hieu-nhie...,Mới nhất,,Không tìm thấy thời gian
4,4911584,Tài tử 75 tuổi đón bạn gái ra tù,https://vnexpress.net/tai-tu-75-tuoi-don-ban-g...,Mới nhất,,Không tìm thấy thời gian
5,4911584,Tài tử 75 tuổi đón bạn gái ra tù,https://vnexpress.net/tai-tu-75-tuoi-don-ban-g...,Mới nhất,,Không tìm thấy thời gian
6,4911630,TP HCM phân cấp hơn 2.100 thủ tục hành chính v...,https://vnexpress.net/tp-hcm-phan-cap-hon-2-10...,Mới nhất,,Không tìm thấy thời gian
7,4911630,TP HCM phân cấp hơn 2.100 thủ tục hành chính v...,https://vnexpress.net/tp-hcm-phan-cap-hon-2-10...,Mới nhất,,Không tìm thấy thời gian
8,4911661,Cháy cư xá 8 người tử vong do 'chập đường dây ...,https://vnexpress.net/chay-cu-xa-8-nguoi-tu-vo...,Mới nhất,,Không tìm thấy thời gian
9,4911661,Cháy cư xá 8 người tử vong do 'chập đường dây ...,https://vnexpress.net/chay-cu-xa-8-nguoi-tu-vo...,Mới nhất,,Không tìm thấy thời gian


In [11]:
data_vnexpress.drop_duplicates(subset=["ID", "Title", "Href"], keep="first", inplace=True)

In [12]:
data_vnexpress

,ID,Title,Href,Category,Content,Time
0,4911600,Cựu phó tổng tham mưu trưởng Nga bị kết án tù ...,https://vnexpress.net/cuu-pho-tong-tham-muu-tr...,Mới nhất,,Không tìm thấy thời gian
2,4911596,200 trẻ có dấu hiệu nhiễm độc chì vì món bánh ...,https://vnexpress.net/200-tre-co-dau-hieu-nhie...,Mới nhất,,Không tìm thấy thời gian
4,4911584,Tài tử 75 tuổi đón bạn gái ra tù,https://vnexpress.net/tai-tu-75-tuoi-don-ban-g...,Mới nhất,,Không tìm thấy thời gian
6,4911630,TP HCM phân cấp hơn 2.100 thủ tục hành chính v...,https://vnexpress.net/tp-hcm-phan-cap-hon-2-10...,Mới nhất,,Không tìm thấy thời gian
8,4911661,Cháy cư xá 8 người tử vong do 'chập đường dây ...,https://vnexpress.net/chay-cu-xa-8-nguoi-tu-vo...,Mới nhất,,Không tìm thấy thời gian
...,...,...,...,...,...,...
851,4910202,"Mua điều khiển tivi Shopee 45 nghìn, công thợ ...",https://vnexpress.net/mua-dieu-khien-tivi-shop...,Ý kiến,,Không tìm thấy thời gian
853,4909690,Viên gạch 'dằn mặt' khi tôi đỗ xe chắn cửa nhà...,https://vnexpress.net/vien-gach-dan-mat-khi-to...,Ý kiến,,Không tìm thấy thời gian
855,4909889,Hành trình tôi 'trưởng thành' sau 5 năm chuyển...,https://vnexpress.net/sap-nhap-tinh-thanh-hanh...,Ý kiến,,Không tìm thấy thời gian
856,4909893,Hành trình sang Mỹ tìm đam mê của một đứa trẻ ...,https://vnexpress.net/hanh-trinh-sang-my-tim-d...,Ý kiến,,Không tìm thấy thời gian
